# Misc. Astronomy Tools

### Stars around a coordinate

In [29]:
# Stars aroudn a coordiante, from GAIA DR2/EDR3
from astropy.coordinates import SkyCoord, Angle
from astroquery.vizier import Vizier

def move_column(df, column_name, insert_position):
    """Move the named column to the posistion in a pandas dataframe"""
    col = df.pop(column_name)
    df.insert(insert_position, col.name, col)
    return df

def search_nearby(ra, dec, catalog_name="I/345/gaia2", radius_arcsec=30, magnitude_limit=21):
    
    c1 = SkyCoord(ra, dec, frame="icrs", unit="deg")
    Vizier.ROW_LIMIT = -1
    result = Vizier.query_region(
        c1,
        catalog=[catalog_name],
        radius=Angle(radius_arcsec, "arcsec"),
    )
    result = result[catalog_name].to_pandas()
    result = result[result.Gmag < magnitude_limit]
    if result.get("RUWE") is not None:
        move_column(result, "RUWE", 7)        
    return result

# catalog_name = "I/345/gaia2"
catalog_name = "I/350/gaiaedr3"
radius_arcsec = 30
magnitude_limit = 20
ra, dec = 263.59892769, 74.47212807

result = search_nearby(ra, dec, catalog_name=catalog_name, radius_arcsec=radius_arcsec, magnitude_limit=magnitude_limit)
print(f"From {catalog_name}:")
result

From I/350/gaiaedr3:


,RA_ICRS,e_RA_ICRS,DE_ICRS,e_DE_ICRS,Source,Plx,e_Plx,RUWE,PM,pmRA,...,loggtemp,PS1,SDSSDR13,SkyMapper2,URAT1,GmagCorr,e_GmagCorr,FGCorr,RAJ2000,DEJ2000
0,263.571972,0.4007,74.474058,0.3897,1655666290100696320,0.4514,0.3970,0.932,2.530,-2.360,...,NaN,197362635720789664,<NA>,<NA>,<NA>,19.915178,0.005791,203.649994,263.572011,74.474062
1,263.609875,0.2231,74.479676,0.2438,1655666290100924672,0.7793,0.2410,1.065,6.726,-3.507,...,NaN,197372636098846400,<NA>,<NA>,823077878,19.098475,0.003905,432.070007,263.609933,74.479651
2,263.593158,0.2527,74.472003,0.2768,1655666294395083264,1.4577,0.2727,1.116,14.053,-0.340,...,NaN,197362635932607232,<NA>,<NA>,<NA>,19.273023,0.005049,367.910004,263.593164,74.472066
3,263.598906,0.0124,74.472164,0.0155,1655666294396195200,1.1199,0.0141,1.144,21.763,3.595,...,3.5,<NA>,<NA>,<NA>,823077874,12.552287,0.002772,179490.000000,263.598846,74.472260
4,263.602235,0.0170,74.479892,0.0207,1655666294396195968,0.2521,0.0190,0.970,15.065,-4.886,...,NaN,197372636022496640,<NA>,<NA>,823077876,14.564581,0.002766,28126.000000,263.602316,74.479828


In [28]:
result.columns

Index(['RA_ICRS', 'e_RA_ICRS', 'DE_ICRS', 'e_DE_ICRS', 'Source', 'Plx',
       'e_Plx', 'RUWE', 'PM', 'pmRA', 'e_pmRA', 'pmDE', 'e_pmDE', 'FG', 'e_FG',
       'Gmag', 'e_Gmag', 'FBP', 'e_FBP', 'BPmag', 'e_BPmag', 'FRP', 'e_FRP',
       'RPmag', 'e_RPmag', 'BP-RP', 'RVDR2', 'e_RVDR2', 'Tefftemp', 'loggtemp',
       'PS1', 'SDSSDR13', 'SkyMapper2', 'URAT1', 'GmagCorr', 'e_GmagCorr',
       'FGCorr', 'RAJ2000', 'DEJ2000'],
      dtype='object')

### Conversion between heliocentric times and barycentric time

In [ ]:
# From https://gist.github.com/StuartLittlefair/4ab7bb8cf21862e250be8cb25f72bb7a

from astropy.coordinates import SkyCoord, EarthLocation
from astropy import units as u
from astropy.time import Time

def helio_to_bary(coords, hjd, obs_name):
    helio = Time(hjd, scale='utc', format='jd')
    obs = EarthLocation.of_site(obs_name)
    star = SkyCoord(coords, unit=(u.hour, u.deg)) 
    ltt = helio.light_travel_time(star, 'heliocentric', location=obs)
    guess = helio - ltt
    # if we assume guess is correct - how far is heliocentric time away from true value?
    delta = (guess + guess.light_travel_time(star, 'heliocentric', obs)).jd  - helio.jd
    # apply this correction
    guess -= delta * u.d

    ltt = guess.light_travel_time(star, 'barycentric', obs)
    return guess.tdb + ltt


def bary_to_helio(coords, bjd, obs_name):
    bary = Time(bjd, scale='tdb', format='jd')
    obs = EarthLocation.of_site(obs_name)
    star = SkyCoord(coords, unit=(u.hour, u.deg))
    ltt = bary.light_travel_time(star, 'barycentric', location=obs) 
    guess = bary - ltt
    delta = (guess + guess.light_travel_time(star, 'barycentric', obs)).jd  - bary.jd
    guess -= delta * u.d
    
    ltt = guess.light_travel    
    

### Time format conversion

In [36]:
from astropy.time import Time
import lightkurve as lk  # for BTJD format

# t1 = Time("2019-05-01", format="iso", scale="utc")
# t1 = Time("58604.4", format="mjd", scale="tdb") # BMJD, from the paper
# t1 = Time("2459307.239419309", format="jd", scale="tdb")  # BJD
# t1 = Time("2032.093", format="btjd")  
t1 = Time(7264.23209 + 2450000, format="jd", scale="utc")  # HJD

print("Source")
print(t1.to_datetime())


t2 = t1.copy()
t2.format = "btjd" 
# potential issue issue: scale is still utc, can't convert to Barycentric

print("Converted:")
print(t2.to_datetime())
t2

Source
2015-08-29 17:34:12.576005
Converted:
2015-08-29 17:34:12.576005


<Time object: scale='utc' format='btjd' value=264.2328791512885>

### Time zone conversion

In [37]:
# adhoc time conversion helper

import pytz
from astropy.time import Time
import lightkurve as lk  # for BTJD format

# t1 = Time("2019-05-01", format="iso", scale="utc")
# t1 = Time("58604.4", format="mjd", scale="tdb") # BMJD, from the paper
t1 = Time(7264.23209 + 2450000, format="jd", scale="utc")  # HJD
# t1 = Time("2459307.239419309", format="jd", scale="tdb")  # BJD
# t1 = Time("2032.093", format="btjd")  # epoch for TIC 284361752, TOI 2294.01 (long period)
dt = t1.to_datetime()
print(dt)
print(pytz.timezone('US/Pacific').fromutc(dt))


2015-08-29 17:34:12.576005
2015-08-29 10:34:12.576005-07:00
